In [2]:
from src.vqe_runner import VQERunner
from src.ansatz_element_sets import UCCSDExcitations
from src.backends import MatrixCacheBackend, QiskitSimBackend
from src.cache import *
from src.molecules.molecules import H2, H4

import logging
import time

/home/jordan/anaconda3/envs/vqe_test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/jordan/anaconda3/envs/vqe_test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

## Create a molecular system

In [10]:
# the unit for radius is an Angstrom
r = 1
molecule = H2(r=r)

## Create a UCCSD ansatz. 

In [11]:
n_orbitals = molecule.n_orbitals
n_electrons = molecule.n_electrons
ansatz_element_type = 'f_exc'
ansatz = UCCSDExcitations(n_orbitals, n_electrons, ansatz_element_type).get_all_elements()


In [18]:
for element in ansatz:
    exc_gen = element.excitation_ge

## Choose a backend to calculate the expectation values for the energy.

Currently the code supports a qiskit noiseless simulator (QiskitSimBackend) and a custom matrix-multiplication noiseless simulator (MatrixCacheBackend).  

In [12]:
backend = QiskitSimBackend

# The MatrixCacheBackend uses a cache to save frequently computed values.
# The cache significantly accelarates the simulation speed, but it also
# requires a lot of RAM. Note that your computer will freeze if it runs out of RAM.
# You can safely use the cache for H2, but for any molecule with more than 
# 10 electrons it will require GBs of RAM

if backend == MatrixCacheBackend:
    global_cache = GlobalCache(molecule)
    global_cache.calculate_exc_gen_sparse_matrices_dict(ansatz)
else: 
    global_cache = None

## Create a VQE runner object

In [13]:
# the code supports all optimizers from scipy.minimize
optimizer = 'Nelder-Mead'
optimizer_options = {'gtol': 10e-8}
vqe_runner = VQERunner(molecule, backend=backend, use_ansatz_gradient=True,
                       optimizer=optimizer, optimizer_options=optimizer_options)

In [14]:
result = vqe_runner.vqe_run(ansatz=ansatz, cache=global_cache)

In [15]:
print(result.fun)

-1.1011503301329544
